In [3]:
import numpy as np 
import pandas as pd 
import json
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt 
import emoji
%matplotlib inline
import nltk
import copy
import random

from transformers import AutoModelWithLMHead, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [39]:
def get_turns(item):
    """
    TODO: Include dummy sentences to cover for "Submit-Deal", "Accept-Deal", "Reject-Deal", "Walk-Away".
    NOTE THAT THIS MIGHT REQUIRE FIXING THE LENGTH ISSUE OF LESS THAN 1024.
    We can work without this probably, unless you are deploying the agent where it has to choose over submit
    accept reject decisions.
    """
    turns = []
    for act in item["acts"]:
        if(act['text'] not in ['Submit-Deal', 'Accept-Deal', 'Reject-Deal', 'Walk-Away', 'Submit-Post-Survey']):
            turns.append(act['text'])
    
    return turns

def get_output(i, item, is_last=False):
    turns = get_turns(item)
    out = " <|endoftext|> ".join(turns)
    if(is_last):
        out += " <|endoftext|>"
    else:
        out += " <|endoftext|>\n"
    
    assert get_token_length(out) <= 1024, i
    return out

def get_token_length(msg):
    ids = tokenizer.encode(msg, return_tensors='pt')
    return len(ids[0])

In [42]:
in_f = "../../../storage/data/all_data-1030-shuffled.json" 
with open(in_f) as f:
    all_data = json.load(f)

dtype = 'test'

if(dtype == 'train'):
    all_data = all_data[:830]
elif(dtype == 'val'):
    all_data = all_data[830:930]
elif(dtype == 'test'):
    all_data = all_data[930:]
else:
    all_data = []
    
print(len(all_data))
print(all_data[0].keys())

outf = "../../../storage/data/hf_" + dtype + ".txt"
print(outf)

with open(outf, "w") as f:
    for i, item in enumerate(all_data):
        if(not i%100):
            print(i)
        output = get_output(i, item, is_last=(i==(len(all_data)-1)))
        f.write(output)

100
dict_keys(['convo_is_finished', 'world_tag', 'bad_workers', 'acts', 'turns', 'workers', 'fpath', 'qualtrics', 'dialogue_id'])
../../../storage/data/hf_test.txt
0


['ðŁĺ', '®', 'ĠðŁĻĤ', 'Ġâĺ', '¹', 'ï¸ı', 'ĠðŁĺ', '¡']